In [1]:
import os
import threading
import multiprocessing
import numpy as np
import tensorflow as tf

from worker import Worker
from ac_network import AC_Network

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
ENV_NAME = 'LunarLander-v2'
STATE_DIM = 8
ACTION_DIM = 4
MONITOR_DIR = './results/' + ENV_NAME

In [3]:
RANDOM_SEED = 1234
LOAD_MODEL = True
TEST_MODEL = False
MODEL_DIR = './model/'
LEARNING_RATE = 0.0001
GAMMA = 0.99

In [ ]:
global master_network
global global_episodes

tf.reset_default_graph()

if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

with tf.device("/cpu:0"):
    np.random.seed(RANDOM_SEED)
    tf.set_random_seed(RANDOM_SEED)

    global_episodes = tf.Variable(0, dtype=tf.int32, name='global_episodes', trainable=False)
    trainer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
    master_network = AC_Network(STATE_DIM, ACTION_DIM, 'global', None)
    num_workers = multiprocessing.cpu_count()

    if TEST_MODEL:
        num_workers = 1

    workers = []
    for i in range(num_workers):
        workers.append(Worker(i, STATE_DIM, ACTION_DIM, trainer, MODEL_DIR, global_episodes,
                              ENV_NAME, RANDOM_SEED, TEST_MODEL))
    saver = tf.train.Saver(max_to_keep=5)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    if LOAD_MODEL or TEST_MODEL:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(MODEL_DIR)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())

    if TEST_MODEL:
        env = workers[0].get_env()
        env.monitor.start(MONITOR_DIR, force=True)
        workers[0].work(GAMMA, sess, coord, saver)
    else:
        worker_threads = []
        for worker in workers:
            worker_work = lambda: worker.work(GAMMA, sess, coord, saver)
            t = threading.Thread(target=(worker_work))
            t.start()
            worker_threads.append(t)
        coord.join(worker_threads)



Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `layer.__call__` method instead.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Loading Model...
INFO:tensorflow:Restoring parameters from ./model/model-0.cptk
Starting worker 0
Starting worker 1Starting worker 3
Starting worker 4
Starting worker 5
Starting worker 8
Starting worker 10

Starting worker 7
Starting worker 11
Starting worker 2
Starting worker 6
Starting worker 9
| Reward: -95.57730562970124  | Episode 0
| Reward: 56.966037677509775  | Episode 1
| Reward: -220.18261897171968 

| Reward: -157.5401528843836  | Episode 101
| Reward: -307.3166667898922  | Episode 102
| Reward: -497.521373227829  | Episode 103
| Reward: -313.3184559543699  | Episode 104
| Reward: -244.12016669067407  | Episode 105
| Reward: -505.79193836206775  | Episode 106
| Reward: -562.5872289315391  | Episode 107
| Reward: -403.41901902994476  | Episode 108
| Reward: -417.5078489211102  | Episode 109
| Reward: -742.2423971594  | Episode 110
| Reward: -506.3022446243127  | Episode 111
| Reward: -766.3260017345631  | Episode 112
| Reward: -424.7695004206292  | Episode 113
| Reward: -480.6775139470428  | Episode 114
| Reward: -681.7389035081382  | Episode 115
| Reward: -488.7323805529567  | Episode 116
| Reward: -422.3243489533991  | Episode 117
| Reward: -362.77322973152013  | Episode 118
| Reward: -372.46608591559794  | Episode 119
| Reward: -374.79724457936953  | Episode 120
| Reward: -490.27017073320735  | Episode 121
| Reward: -368.12170922177245  | Episode 122
| Reward: -288.6574107794689

| Reward: -121.17869646969783  | Episode 286
| Reward: -115.02873164630824  | Episode 287
| Reward: -259.7109665847762  | Episode 288
| Reward: -148.66973461274443  | Episode 289
| Reward: -426.19692440662675  | Episode 290
| Reward: -369.96488455742866  | Episode 291
| Reward: -232.08053999736234  | Episode 292
| Reward: -578.5144275586229  | Episode 293
| Reward: -435.17660317304785  | Episode 294
| Reward: -240.4543362506779  | Episode 295
| Reward: -339.3660455238714  | Episode 296
| Reward: -149.6220724738991  | Episode 297
| Reward: -325.5820339149357  | Episode 298
| Reward: -340.8175220760778  | Episode 299
| Reward: -545.3920039997583  | Episode 300
| Reward: -225.43050742292212  | Episode 301
| Reward: -363.6777937168828  | Episode 302
| Reward: -131.04280419969425  | Episode 303
| Reward: -425.52777038558406  | Episode 304
| Reward: -407.7416765933781  | Episode 305
| Reward: -524.3923314346201  | Episode 306
| Reward: -435.2501169473733  | Episode 307
| Reward: -474.7249099

| Reward: -321.1936817228881  | Episode 471
| Reward: -401.37698703188437  | Episode 472
| Reward: -441.89493252394846  | Episode 473
| Reward: -369.97169267660485  | Episode 474
| Reward: -433.9030612645802  | Episode 475
| Reward: -291.6232884937781  | Episode 476
| Reward: -396.21724665371175  | Episode 477
| Reward: -236.81160341298747  | Episode 478
| Reward: -291.3150213242859  | Episode 479
| Reward: -249.3955610679711  | Episode 480
| Reward: -490.8230863059363  | Episode 481
| Reward: -464.0904337556225  | Episode 482
| Reward: -529.9664501126382  | Episode 483
| Reward: -368.21370589005267  | Episode 484
| Reward: -197.1434442493844  | Episode 485
| Reward: -186.28868207795438  | Episode 486
| Reward: -406.7714142213102  | Episode 487
| Reward: -107.86756755069867  | Episode 488
| Reward: -426.4442904702032  | Episode 489
| Reward: -166.81816588577004  | Episode 490
| Reward: -428.2552917532916  | Episode 491
| Reward: -331.61374901264037  | Episode 492
| Reward: -559.4954207

| Reward: -535.9459952580133  | Episode 649
| Reward: -332.96775780097016  | Episode 650
| Reward: -397.8763017634095  | Episode 651
| Reward: -430.3005559996153  | Episode 652
| Reward: -213.0935131889056  | Episode 653
| Reward: -115.83801838028334  | Episode 654
| Reward: -265.3281778004449  | Episode 655
| Reward: -455.4357264330369  | Episode 656
| Reward: -569.2177022858341  | Episode 657
| Reward: -351.7067777275478  | Episode 658
| Reward: -487.6074170944704  | Episode 659
| Reward: -441.3525289189223  | Episode 660
| Reward: -185.2265077236435  | Episode 661
| Reward: -621.9313122148427  | Episode 662
| Reward: -253.72939520127164  | Episode 663
| Reward: -532.8125210461614  | Episode 664
| Reward: -414.309067112621  | Episode 665
| Reward: -607.9104323795555  | Episode 666
| Reward: -606.0005745347787  | Episode 667
| Reward: -525.669373510988  | Episode 668
| Reward: -461.1247140562891  | Episode 669
| Reward: -573.2330654950108  | Episode 670
| Reward: -629.7764200934929  |

...

| Reward: -597.14310938275  | Episode 2479
| Reward: -435.0507415632846  | Episode 2480
| Reward: -808.505996945483  | Episode 2481
| Reward: -338.0841061520441  | Episode 2482
| Reward: -243.13189808747788  | Episode 2483
| Reward: -311.01989349823805  | Episode 2484
| Reward: -480.2526513108148  | Episode 2485
| Reward: -254.1020319658249  | Episode 2486
| Reward: -406.67230204773944  | Episode 2487
| Reward: -347.52541755189606  | Episode 2488
| Reward: -276.24662129052786  | Episode 2489
| Reward: -354.4770577212041  | Episode 2490
| Reward: -580.8765359780889  | Episode 2491
| Reward: -452.55368662164364  | Episode 2492
| Reward: -324.3924013195137  | Episode 2493
| Reward: -214.07388112392817  | Episode 2494
| Reward: -693.6684685595692  | Episode 2495
| Reward: -458.0587927062673  | Episode 2496
| Reward: -771.2784591469823  | Episode 2497
| Reward: -467.9495805189558  | Episode 2498
| Reward: -175.64614736763843  | Episode 2499
| Reward: -694.9100276273995  | Episode 2500
| Rew

| Reward: -414.8223439692012  | Episode 2661
| Reward: -720.0639116667841  | Episode 2662
| Reward: -670.3135492208464  | Episode 2663
| Reward: -631.5230312710352  | Episode 2664
| Reward: -657.6683955627198  | Episode 2665
| Reward: -636.0274631559599  | Episode 2666
| Reward: -370.78978605351415  | Episode 2667
| Reward: -646.9333039468165  | Episode 2668
| Reward: -642.57076824684  | Episode 2669
| Reward: -652.3892065857854  | Episode 2670
| Reward: -729.8011790546744  | Episode 2671
| Reward: -296.06194267196815  | Episode 2672
| Reward: -703.2167307898109  | Episode 2673
| Reward: -104.33210604420437  | Episode 2674
| Reward: -191.91958518248885  | Episode 2675
| Reward: -515.2056440081149  | Episode 2676
| Reward: -701.2042954162963  | Episode 2677
| Reward: -751.4802122308146  | Episode 2678
| Reward: -150.39381281583724  | Episode 2679
| Reward: -706.7639035213315  | Episode 2680
| Reward: -964.5034363251589  | Episode 2681
| Reward: -287.41259737842097  | Episode 2682
| Rewa

| Reward: -157.96552625852178  | Episode 2842
| Reward: -294.54128224961653  | Episode 2843
| Reward: -70.6455726813283  | Episode 2844
| Reward: -303.79617850062124  | Episode 2845
| Reward: -272.8170575514588  | Episode 2846
| Reward: -351.6736380400098  | Episode 2847
| Reward: -295.2223461445236  | Episode 2848
| Reward: -460.14977670869337  | Episode 2849
| Reward: -394.0945230462226  | Episode 2850
| Reward: -364.5737989863765  | Episode 2851
| Reward: -388.35582833643366  | Episode 2852
| Reward: -325.3036247881779  | Episode 2853
| Reward: -102.80859504490712  | Episode 2854
| Reward: -346.9540129254093  | Episode 2855
| Reward: -311.83326146571284  | Episode 2856
| Reward: -200.99775087247116  | Episode 2857
| Reward: -115.89519138324557  | Episode 2858
| Reward: -138.49441095746366  | Episode 2859
| Reward: -96.5118362168551  | Episode 2860
| Reward: -129.96921023316096  | Episode 2861
| Reward: -22.658787411701212  | Episode 2862
| Reward: -76.67010480762039  | Episode 2863


| Reward: -8.510696685976484  | Episode 3023
| Reward: -305.29035430446254  | Episode 3024
| Reward: -239.42846781176965  | Episode 3025
| Reward: -72.84917442804615  | Episode 3026
| Reward: -62.274491029467015  | Episode 3027
| Reward: 4.287122576086034  | Episode 3028
| Reward: -52.434707543616625  | Episode 3029
| Reward: -134.49158763186614  | Episode 3030
| Reward: -318.92618391458893  | Episode 3031
| Reward: -114.81219483829148  | Episode 3032
| Reward: -134.3503534558023  | Episode 3033
| Reward: -144.6366674358415  | Episode 3034
| Reward: -87.62226287325582  | Episode 3035
| Reward: -69.25885933369761  | Episode 3036
| Reward: -99.56397240654232  | Episode 3037
| Reward: -146.6202009781563  | Episode 3038
| Reward: -159.5812881418462  | Episode 3039
| Reward: -136.48169476467365  | Episode 3040
| Reward: -106.12563840432387  | Episode 3041
| Reward: -134.94175209279095  | Episode 3042
| Reward: -106.52153487410055  | Episode 3043
| Reward: -130.80987682229716  | Episode 3044

| Reward: 34.700523177726836  | Episode 3204
| Reward: -117.35472903870803  | Episode 3205
| Reward: -86.95251767392116  | Episode 3206
| Reward: -25.030283269493836  | Episode 3207
| Reward: -106.27499830487773  | Episode 3208
| Reward: -63.14642460360779  | Episode 3209
| Reward: -9.125571145186655  | Episode 3210
| Reward: -53.85365154649905  | Episode 3211
| Reward: -1.9144599356190497  | Episode 3212
| Reward: -99.21801001760238  | Episode 3213
| Reward: -102.61363633153147  | Episode 3214
| Reward: -78.59531717353039  | Episode 3215
| Reward: -63.90642072410983  | Episode 3216
| Reward: -114.66198126342294  | Episode 3217
| Reward: -134.04858264810292  | Episode 3218
| Reward: -40.562386318167285  | Episode 3219
| Reward: 6.208810742171181  | Episode 3220
| Reward: -127.62574096657765  | Episode 3221
| Reward: -189.04667333181555  | Episode 3222
| Reward: -104.05676089693057  | Episode 3223
| Reward: -53.439477934262015  | Episode 3224
| Reward: -232.93903127592313  | Episode 322